In [99]:
import gym
import matplotlib.pyplot as plt
import numpy as np
import random
import sys

from collections import defaultdict

%matplotlib inline

env = gym.make('Blackjack-v0')

In [100]:
def muestrea_politica(Q, estado,epsilon):
    if random.random() < epsilon: 
        accion = env.action_space.sample()
    else:
        if Q[estado,1]>Q[estado,0]: accion = 1
        else: accion = 0
            
    return accion

In [101]:
def inicializa_Q():
    estados = list()
    for mis_puntos in range(11,22):
        for sus_puntos in range(1,11):
            for utilizable in range(0,2):
                estados.append((mis_puntos,sus_puntos,utilizable))

    M = {}
    for estado in estados:
        mis_puntos, puntos_del_repartidor, as_utilizable = estado
        if (mis_puntos < 20):
            M[(estado,0)] = -0.001
            M[(estado,1)] = 0.001   # favorece pedir
        else:
            M[(estado,0)] = 0.001  # favorece quedarse
            M[(estado,1)] = -0.001
    return M

In [ ]:
key = ''


In [115]:
def sarsa(ambiente, num_episodios, gama,wins_loses):

    lr = 0.8
    cuenta_retornos = defaultdict(float)
    
    Q = inicializa_Q()
    
    for i in range(0, num_episodios+1):
        
        if i%100000==0 and i>0: print("Núm. episodios: "+str(i))
        
        lista_estados_accion = list()
        
        estado = env.reset()
        mis_puntos, _, as_utilizable = estado 
        valor = mis_puntos if as_utilizable==0 else mis_puntos+10

        while valor < 12: #mientras no llegue a 12, sigo pidiendo cartas
            estado, recompenza, termino, _ = env.step(1)
            mis_puntos, _, as_utilizable = estado 
            valor = mis_puntos if as_utilizable==0 else mis_puntos+10

        
        accion = np.argmax(np.array([Q[estado,1],Q[estado,0]]) + np.random.randn(1,env.action_space.n)*(1./(i+1)))
        #utilizando epsilon greedy elegimos accion     

        lista_estados_accion.append((estado,accion)) 
        s1, recompenza, termino, _ = env.step(accion)
        # elegimos s' 
        while not termino:
            #Actualizamos en cada episodio
            a1 = np.argmax(np.array([Q[s1,1],Q[s1,0]]) + np.random.randn(1,env.action_space.n)*(1./(i+1))) #elegimos a' utilizando s'
            Q[estado,accion] = Q[estado,accion] + lr*(recompenza + gama*Q[s1,a1] - Q[estado,accion]) #actualizamos valor del estado y la acción
            estado = s1
            accion = a1
            lista_estados_accion.append((estado,accion))   
            s1, recompenza, termino, _ = env.step(accion) 
        wins_loses['total'] +=1

        key = 'win' if recompenza> 0 else 'lose'
        wins_loses[key]+=1
                            
    return Q

In [116]:
wl= dict.fromkeys(['total','win','lose'],0)
Q_10k = sarsa(env, num_episodios=10000000,gama=0.95,wins_loses = wl)

Núm. episodios: 100000
Núm. episodios: 200000
Núm. episodios: 300000
Núm. episodios: 400000
Núm. episodios: 500000
Núm. episodios: 600000
Núm. episodios: 700000
Núm. episodios: 800000
Núm. episodios: 900000
Núm. episodios: 1000000
Núm. episodios: 1100000
Núm. episodios: 1200000
Núm. episodios: 1300000
Núm. episodios: 1400000
Núm. episodios: 1500000
Núm. episodios: 1600000
Núm. episodios: 1700000
Núm. episodios: 1800000
Núm. episodios: 1900000
Núm. episodios: 2000000
Núm. episodios: 2100000
Núm. episodios: 2200000
Núm. episodios: 2300000
Núm. episodios: 2400000
Núm. episodios: 2500000
Núm. episodios: 2600000
Núm. episodios: 2700000
Núm. episodios: 2800000
Núm. episodios: 2900000
Núm. episodios: 3000000
Núm. episodios: 3100000
Núm. episodios: 3200000
Núm. episodios: 3300000
Núm. episodios: 3400000
Núm. episodios: 3500000
Núm. episodios: 3600000
Núm. episodios: 3700000
Núm. episodios: 3800000
Núm. episodios: 3900000
Núm. episodios: 4000000
Núm. episodios: 4100000
Núm. episodios: 4200000
N

In [96]:
def imprime_politica(Q):
    print('---- Política ----')
    for useable in [0, 1]:
        if useable:
            print('As utilizable')
        else:
            print('As no utilizable')
        for val in range(21,10,-1):
            for card in range(1,11):
                if (Q[((val,card,useable),1)] > Q[((val,card,useable),0)]):
                    print('X',end="")
                else:
                    print(' ',end="")
            print('| %d' % val)
        print("A2345678910")
        print(' ')
    
imprime_politica(Q_10k)

---- Política ----
As no utilizable
          | 21
          | 20
XXXXXXX X | 19
XXXX XXXXX| 18
 X XX   XX| 17
X X  XXXX | 16
X X XX  X | 15
 X X  XX  | 14
 X   X  X | 13
 XX    XX | 12
XXXXXXXXXX| 11
A2345678910
 
As utilizable
          | 21
          | 20
 XX XXXXXX| 19
XXXXX X X | 18
   XX X X | 17
XXXXXXXX X| 16
XXXXXXXXX | 15
XXXXXXXXX | 14
XXXXXXXXXX| 13
XXXXXXXXX | 12
XXXXXXXXXX| 11
A2345678910
 


In [121]:
n = wl['total']
for k in wl.keys():
    wl[k] = (wl[k]/n)*100

In [122]:
wl

{'total': 100.0, 'win': 27.568857243114277, 'lose': 72.43114275688572}